# Download toy seismic dataset 

This first notebook downloads a small toy dataset to test the package and show the main steps toward a simple earthquake detection and location. The dataset consists of one day of data from eight broadband seismic stations with three components. The seismic stations are located in the North Anatolian fault zone. 

**Note** : this notebook uses the `mass_downloader` of `obspy` to find and download the data from the IRIS data center. The download duration can change depending on your location, do not hesitate to consider another dataset that would suit you better. A more extensive tutorial about the mass downloader [is available in obspy's documentation](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.mass_downloader.html).

In [2]:
import obspy
import os

from obspy.clients.fdsn import mass_downloader

## Download destination path

By default, the package comes with a `data/` folder which can be used to store the downloaded waveforms. The waveforms downloaded there should not be pushed to a new branch if you want to contribute. We indicated the `.mseed` format in the `.gitignore` file; if you download any other data format, please update `.gitignore` accordingly. Feel free to change it to any convenient path of your choice. 

In [10]:
DIRPATH_DESTINATION = "../data/"

In [11]:
os.makedirs(DIRPATH_DESTINATION, exist_ok=True)

## Prepare the downloader

We here define the geographical domain and other restrictions such as data quality and temporal coverage for downwload. We restrict the download to the North Anatolian domain thanks to the `RectangularDomain` function, and we select the data from the eight seismic stations on either channels `HH` or `BH` with available continuous data over one day (2012/07/26) within the `Restrictions` function.

In [14]:
# Geographical restrictions
domain = mass_downloader.RectangularDomain(
    minlatitude=40.60,
    maxlatitude=40.76,
    minlongitude=30.20,
    maxlongitude=30.44,
)

# Time and station restrictions
restrictions = mass_downloader.Restrictions(
    starttime=obspy.UTCDateTime("2012-07-26"),
    endtime=obspy.UTCDateTime("2012-07-27"),
    chunklength_in_sec=86400.0,
    network="YH",
    location="*",
    channel="BH*,HH*",
    station="SAUV,SPNC,DC08,DC07,DC06,DD06,DE07,DE08",
    reject_channels_with_gaps=False,
    minimum_length=0.0,
    minimum_interstation_distance_in_m=100.0,
    channel_priorities=["HH[ZNE]", "BH[ZNE]"],
)

# Downloader instance
downloader = mass_downloader.MassDownloader(providers=["IRIS"])

[2022-08-04 08:01:20,575] - obspy.clients.fdsn.mass_downloader - INFO: Initializing FDSN client(s) for IRIS.
[2022-08-04 08:01:20,575] - obspy.clients.fdsn.mass_downloader - INFO: Initializing FDSN client(s) for IRIS.
[2022-08-04 08:01:20,584] - obspy.clients.fdsn.mass_downloader - INFO: Successfully initialized 1 client(s): IRIS.
[2022-08-04 08:01:20,584] - obspy.clients.fdsn.mass_downloader - INFO: Successfully initialized 1 client(s): IRIS.


## Download

Download the data. This can take several minutes depending on the internet connection and the machine.

In [9]:
downloader.download(
    domain,
    restrictions,
    mseed_storage=DIRPATH_DESTINATION,
    stationxml_storage=DIRPATH_DESTINATION,
)

[2022-08-02 17:13:55,167] - obspy.clients.fdsn.mass_downloader - INFO: Total acquired or preexisting stations: 0
[2022-08-02 17:13:55,170] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Requesting reliable availability.
[2022-08-02 17:13:56,359] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Successfully requested availability (1.19 seconds)
[2022-08-02 17:13:56,363] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Found 8 stations (24 channels).
[2022-08-02 17:13:56,368] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Will attempt to download data from 8 stations.
[2022-08-02 17:13:56,392] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Status for 24 time intervals/channels before downloading: NEEDS_DOWNLOADING
[2022-08-02 17:14:04,817] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Successfully downloaded 3 channels (of 3)
[2022-08-02 17:14:09,067] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS

{'IRIS': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x2aac1a2b42e0>}